In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# Remove loan status target from features data
y = np.array(loans_df.Loan_Status_Fully_Paid)
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.849


In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/50
27317/27317 [==============================] - 2s 90us/sample - loss: 0.4350 - accuracy: 0.8220
Epoch 2/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3858 - accuracy: 0.8492
Epoch 3/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3823 - accuracy: 0.8493
Epoch 4/50
27317/27317 [==============================] - 1s 53us/sample - loss: 0.3803 - accuracy: 0.8493
Epoch 5/50
27317/27317 [==============================] - 1s 54us/sample - loss: 0.3791 - accuracy: 0.8492
Epoch 6/50
27317/27317 [==============================] - 1s 53us/sample - loss: 0.3780 - accuracy: 0.8495
Epoch 7/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3775 - accuracy: 0.8495
Epoch 8/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3765 - accuracy: 0.8496
Epoch 9/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3756 - accuracy: 0.8495
Epoch 10/50
27

In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/100
  864/27317 [..............................] - ETA: 23s - loss: 0.3710 - accuracy: 0.8461 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1888/27317 [=>............................] - ETA: 11s - loss: 0.3779 - accuracy: 0.8438
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2528/27317 [=>............................] - ETA: 8s - loss: 0.3712 - accuracy: 0.8457 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3264/27317 [==>...........................] - ETA: 7s - loss: 0.3686 - accuracy: 0.8471
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4960/27317 [====>.........................] - ETA: 4s - loss: 0.3627 - accuracy: 0.8488
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5696/27317 [=====>........................] - ETA: 4s - loss: 0.3648 - accuracy: 0.8480
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6368/27317 [=====>........................] - ETA: 3s - loss: 0.3671 - accuracy: 0.8

 1600/27317 [>.............................] - ETA: 1s - loss: 0.3570 - accuracy: 0.8487
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 2304/27317 [=>............................] - ETA: 1s - loss: 0.3567 - accuracy: 0.8524
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 3040/27317 [==>...........................] - ETA: 1s - loss: 0.3526 - accuracy: 0.8553
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4576/27317 [====>.........................] - ETA: 1s - loss: 0.3532 - accuracy: 0.8545
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5184/27317 [====>.........................] - ETA: 1s - loss: 0.3554 - accuracy: 0.8522
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5920/27317 [=====>........................] - ETA: 1s - loss: 0.3547 - accuracy: 0.8530
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7616/27317 [=======>......................] - ETA: 1s - loss: 0.3590 - accuracy: 0.8523
Epoch 00003: saving model to checkp

 3200/27317 [==>...........................] - ETA: 1s - loss: 0.3523 - accuracy: 0.8525
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 3904/27317 [===>..........................] - ETA: 1s - loss: 0.3532 - accuracy: 0.8535
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 4608/27317 [====>.........................] - ETA: 1s - loss: 0.3501 - accuracy: 0.8548
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 6272/27317 [=====>........................] - ETA: 1s - loss: 0.3571 - accuracy: 0.8511
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 6912/27317 [======>.......................] - ETA: 1s - loss: 0.3571 - accuracy: 0.8510
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 8288/27317 [========>.....................] - ETA: 1s - loss: 0.3527 - accuracy: 0.8545
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 9024/27317 [========>.....................] - ETA: 1s - loss: 0.3527 - accuracy: 0.8548
Epoch 00005: saving model to checkp

 4544/27317 [===>..........................] - ETA: 1s - loss: 0.3732 - accuracy: 0.8424
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 5312/27317 [====>.........................] - ETA: 1s - loss: 0.3692 - accuracy: 0.8456
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 6880/27317 [======>.......................] - ETA: 1s - loss: 0.3613 - accuracy: 0.8512
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 7584/27317 [=======>......................] - ETA: 1s - loss: 0.3631 - accuracy: 0.8507
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 8992/27317 [========>.....................] - ETA: 1s - loss: 0.3615 - accuracy: 0.8531
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 9728/27317 [=========>....................] - ETA: 1s - loss: 0.3627 - accuracy: 0.8522
Epoch 00007: saving model to checkpoints/weights.07.hdf5
10432/27317 [==========>...................] - ETA: 1s - loss: 0.3611 - accuracy: 0.8528
Epoch 00007: saving model to checkp

 6624/27317 [======>.......................] - ETA: 1s - loss: 0.3534 - accuracy: 0.8546
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 7392/27317 [=======>......................] - ETA: 1s - loss: 0.3539 - accuracy: 0.8547
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 8032/27317 [=======>......................] - ETA: 1s - loss: 0.3544 - accuracy: 0.8545
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 9600/27317 [=========>....................] - ETA: 1s - loss: 0.3537 - accuracy: 0.8549
Epoch 00009: saving model to checkpoints/weights.09.hdf5
10240/27317 [==========>...................] - ETA: 1s - loss: 0.3522 - accuracy: 0.8562
Epoch 00009: saving model to checkpoints/weights.09.hdf5
11424/27317 [===========>..................] - ETA: 1s - loss: 0.3511 - accuracy: 0.8567
Epoch 00009: saving model to checkpoints/weights.09.hdf5
12128/27317 [============>.................] - ETA: 1s - loss: 0.3519 - accuracy: 0.8560
Epoch 00009: saving model to checkp

...

15968/27317 [================>.............] - ETA: 0s - loss: 0.3482 - accuracy: 0.8559
Epoch 00091: saving model to checkpoints/weights.91.hdf5
17376/27317 [==================>...........] - ETA: 0s - loss: 0.3489 - accuracy: 0.8560
Epoch 00091: saving model to checkpoints/weights.91.hdf5
18112/27317 [==================>...........] - ETA: 0s - loss: 0.3481 - accuracy: 0.8564
Epoch 00091: saving model to checkpoints/weights.91.hdf5
18848/27317 [===================>..........] - ETA: 0s - loss: 0.3486 - accuracy: 0.8566
Epoch 00091: saving model to checkpoints/weights.91.hdf5
20384/27317 [=====================>........] - ETA: 0s - loss: 0.3489 - accuracy: 0.8565
Epoch 00091: saving model to checkpoints/weights.91.hdf5
21088/27317 [======================>.......] - ETA: 0s - loss: 0.3489 - accuracy: 0.8566
Epoch 00091: saving model to checkpoints/weights.91.hdf5
22432/27317 [=======================>......] - ETA: 0s - loss: 0.3490 - accuracy: 0.8567
Epoch 00091: saving model to checkp

17472/27317 [==================>...........] - ETA: 0s - loss: 0.3495 - accuracy: 0.8546
Epoch 00093: saving model to checkpoints/weights.93.hdf5
18976/27317 [===================>..........] - ETA: 0s - loss: 0.3494 - accuracy: 0.8551
Epoch 00093: saving model to checkpoints/weights.93.hdf5
19712/27317 [====================>.........] - ETA: 0s - loss: 0.3486 - accuracy: 0.8557
Epoch 00093: saving model to checkpoints/weights.93.hdf5
20448/27317 [=====================>........] - ETA: 0s - loss: 0.3486 - accuracy: 0.8559
Epoch 00093: saving model to checkpoints/weights.93.hdf5
22176/27317 [=======================>......] - ETA: 0s - loss: 0.3484 - accuracy: 0.8561
Epoch 00093: saving model to checkpoints/weights.93.hdf5
22688/27317 [=======================>......] - ETA: 0s - loss: 0.3484 - accuracy: 0.8562
Epoch 00093: saving model to checkpoints/weights.93.hdf5
24128/27317 [=========================>....] - ETA: 0s - loss: 0.3487 - accuracy: 0.8561
Epoch 00093: saving model to checkp

19040/27317 [===================>..........] - ETA: 0s - loss: 0.3512 - accuracy: 0.8572
Epoch 00095: saving model to checkpoints/weights.95.hdf5
20736/27317 [=====================>........] - ETA: 0s - loss: 0.3522 - accuracy: 0.8560
Epoch 00095: saving model to checkpoints/weights.95.hdf5
21376/27317 [======================>.......] - ETA: 0s - loss: 0.3514 - accuracy: 0.8561
Epoch 00095: saving model to checkpoints/weights.95.hdf5
22112/27317 [=======================>......] - ETA: 0s - loss: 0.3509 - accuracy: 0.8563
Epoch 00095: saving model to checkpoints/weights.95.hdf5
23648/27317 [========================>.....] - ETA: 0s - loss: 0.3496 - accuracy: 0.8573
Epoch 00095: saving model to checkpoints/weights.95.hdf5
24384/27317 [=========================>....] - ETA: 0s - loss: 0.3496 - accuracy: 0.8574
Epoch 00095: saving model to checkpoints/weights.95.hdf5
25824/27317 [===========================>..] - ETA: 0s - loss: 0.3490 - accuracy: 0.8574
Epoch 00095: saving model to checkp

21312/27317 [======================>.......] - ETA: 0s - loss: 0.3517 - accuracy: 0.8552
Epoch 00097: saving model to checkpoints/weights.97.hdf5
22016/27317 [=======================>......] - ETA: 0s - loss: 0.3514 - accuracy: 0.8559
Epoch 00097: saving model to checkpoints/weights.97.hdf5
22784/27317 [========================>.....] - ETA: 0s - loss: 0.3515 - accuracy: 0.8560
Epoch 00097: saving model to checkpoints/weights.97.hdf5
24448/27317 [=========================>....] - ETA: 0s - loss: 0.3503 - accuracy: 0.8569
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25184/27317 [==========================>...] - ETA: 0s - loss: 0.3499 - accuracy: 0.8571
Epoch 00097: saving model to checkpoints/weights.97.hdf5
26560/27317 [============================>.] - ETA: 0s - loss: 0.3493 - accuracy: 0.8578
Epoch 00097: saving model to checkpoints/weights.97.hdf5
27317/27317 [==============================] - 2s 71us/sample - loss: 0.3495 - accuracy: 0.8575
Epoch 98/100
   32/27317 [..

22816/27317 [========================>.....] - ETA: 0s - loss: 0.3469 - accuracy: 0.8596
Epoch 00099: saving model to checkpoints/weights.99.hdf5
24032/27317 [=========================>....] - ETA: 0s - loss: 0.3475 - accuracy: 0.8594
Epoch 00099: saving model to checkpoints/weights.99.hdf5
24512/27317 [=========================>....] - ETA: 0s - loss: 0.3476 - accuracy: 0.8591
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25952/27317 [===========================>..] - ETA: 0s - loss: 0.3486 - accuracy: 0.8585
Epoch 00099: saving model to checkpoints/weights.99.hdf5
26624/27317 [============================>.] - ETA: 0s - loss: 0.3486 - accuracy: 0.8585
Epoch 00099: saving model to checkpoints/weights.99.hdf5
27317/27317 [==============================] - 2s 79us/sample - loss: 0.3496 - accuracy: 0.8579
Epoch 100/100
  800/27317 [..............................] - ETA: 1s - loss: 0.3111 - accuracy: 0.8788
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 1472/27317 [

In [38]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


KeyError: 0

In [33]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [34]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Error when checking input: expected dense_9_input to have shape (27317,) but got array with shape (36,)